# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("weather_analysed.csv", encoding="utf-8")
data.head()

,Unnamed: 0,City,Date,Cloudiness,Country,Wind Speed,Humidity,Latitude,Longitude,Max Temp
0,0,Kandi,1616240464,91,BJ,3.94,25,11.1342,2.9386,96.24
1,1,Alghero,1616240466,40,IT,17.27,62,40.5589,8.3181,53.60
2,2,Mahébourg,1616240468,75,MU,12.66,83,-20.4081,57.7000,82.40
3,3,Yangambi,1616240471,58,CD,2.33,38,0.8102,24.4336,95.79
4,4,Toropets,1616240472,86,RU,7.31,61,56.4978,31.6353,28.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
Humidity = data["Humidity"].astype(float)
max_humidity = Humidity.max()
locations = data[["Latitude", "Longitude"]]

In [4]:
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, 
                                 max_intensity = max_humidity,point_radius=3)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
data_df = data.copy()
data_df = data_df[data_df['Max Temp'] >= 70]
data_df = data_df[data_df['Max Temp'] <= 80]  
data_df = data_df[data_df['Wind Speed'] <= 10]
data_df = data_df[data_df['Cloudiness'] == 0]
data_df = data_df.dropna()
data_df = data_df.reset_index()
data_df

,index,Unnamed: 0,City,Date,Cloudiness,Country,Wind Speed,Humidity,Latitude,Longitude,Max Temp
0,104,104,Boa Vista,1616240699,0,BR,5.75,73,2.8197,-60.6733,75.20
1,173,173,El Salvador,1616240698,0,SV,2.30,69,13.8333,-88.9167,77.00
2,204,204,Chenghai,1616240924,0,CN,6.71,69,23.4632,116.7680,78.80
3,256,256,Tūkrah,1616241032,0,LY,7.07,45,32.5341,20.5791,71.28
4,320,320,Formosa,1616241172,0,AR,8.05,88,-26.1775,-58.1781,77.00
5,434,434,Lázaro Cárdenas,1616241412,0,MX,4.05,60,17.9583,-102.2000,72.03
6,455,455,Pochutla,1616241462,0,MX,4.00,75,15.7432,-96.4661,71.01
7,533,533,La Orilla,1616241687,0,MX,3.87,59,17.9833,-102.2333,71.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_count = []

for i in range(len(data_df)):
    lat_hotel = data_df.loc[i]['Latitude']
    lng_hotel = data_df.loc[i]['Longitude']

    params = {"location": f"{lat_hotel},{lng_hotel}", "radius": 5000, "types" : "hotel", "key": g_key}
    
    hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request_data = requests.get(hotel_url, params=params)
    jsn = request_data.json()
    try:
        hotel_count.append(jsn['results'][0]['name'])
    except:
        hotel_count.append("")

In [7]:
data_df["Hotel Name"] = hotel_count
hotel_df = data_df
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,index,Unnamed: 0,City,Date,Cloudiness,Country,Wind Speed,Humidity,Latitude,Longitude,Max Temp,Hotel Name
0,104,104,Boa Vista,1616240699,0,BR,5.75,73,2.8197,-60.6733,75.20,Boa Vista
1,173,173,El Salvador,1616240698,0,SV,2.30,69,13.8333,-88.9167,77.00,Tejutepeque
2,204,204,Chenghai,1616240924,0,CN,6.71,69,23.4632,116.7680,78.80,Shantou
3,256,256,Tūkrah,1616241032,0,LY,7.07,45,32.5341,20.5791,71.28,Tocra
4,320,320,Formosa,1616241172,0,AR,8.05,88,-26.1775,-58.1781,77.00,Formosa


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))